In [17]:
import chess
import chess.engine
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from chess_env import ChessEnv

2024-02-02 20:59:48.473777: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 20:59:48.510364: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 20:59:48.510388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 20:59:48.511378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 20:59:48.516998: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 20:59:48.517676: I tensorflow/core/platform/cpu_feature_guard.cc:1

Observation: [[[0 0 0 1 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 1]
  [0 0 1 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 1 0 0]]

 [[1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [0 0 0 0 1 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [0 0 0 0 0 0]
  [1 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [1 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 1 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [1 0 0 0 0 0]
  [0 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 

In [18]:
def build_model(input_shape, num_actions):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_actions, activation='linear')
    ])
    return model

In [21]:
input_shape = (8, 8, 6)
num_actions = 4096
learning_rate = 0.001
discount_factor = 0.99

#create DNQ model
model = build_model(input_shape, num_actions)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss='mean_squared_error'
)
env = ChessEnv()
num_episodes = 1000

for episode in range(num_episodes):
    state = env.reset()
    state = np.expand_dims(state, axis=0)

    total_reward = 0
    done = False
    while not done: 
        # select actioi unsit epsilon-greedy policy
        action = np.random.choice(num_actions)
        next_state, reward, done, info = env.step(action)
        next_staate = np.expand_dims(next_state, axis=0)

        target = reward + discount_factor * np.max(model.predict(next_state))
        with tf.GradientTape() as tape:
            q_values = model(state, training=True)
            action_mask = tf.one_hot(action, num_actions)
            q_values = tf.reduce_sum(tf.multiply(q_values, action_mask), axis=1)
            loss = keras.losses.mean_squared_error(target, q_values)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        total_reward += reward
        state = next_state
    print(f'Episode: {episode}, total_reward: {total_reward}')



1/1 [==============================] - 0s 30ms/step


KeyboardInterrupt: 

In [10]:
model.save('model.h5')

[Move.from_uci('g8e7'), Move.from_uci('g8h6'), Move.from_uci('g8f6'), Move.from_uci('e8f8'), Move.from_uci('e8e7'), Move.from_uci('d8c8'), Move.from_uci('d8e7'), Move.from_uci('d8d7'), Move.from_uci('d8f6'), Move.from_uci('d8d6'), Move.from_uci('b8d7'), Move.from_uci('b8c6'), Move.from_uci('b8a6'), Move.from_uci('a8a7'), Move.from_uci('a8a6'), Move.from_uci('g7f8'), Move.from_uci('g7h6'), Move.from_uci('g7f6'), Move.from_uci('g5h4'), Move.from_uci('e5d4'), Move.from_uci('d5c4'), Move.from_uci('h7h6'), Move.from_uci('f7f6'), Move.from_uci('c7c6'), Move.from_uci('b7b6'), Move.from_uci('g5g4'), Move.from_uci('e5e4'), Move.from_uci('a5a4'), Move.from_uci('h7h5'), Move.from_uci('f7f5'), Move.from_uci('c7c5'), Move.from_uci('b7b5')]
